# 課題
## 課題1：テストデータの作成

テストデータを以下のように作成する。

- 日付：2013/1/1 ~ 2016/6/30
- 店舗ID：$[1,2,\dots,100]$
- 商品ID：$[1,2, \dots, 1000]$

- 売上高：区間$[0, 10000]$の一様分布に従う乱数
- 販売数量：$\left\{ 0,1,2, \dots, 10000 \right\}$からのランダムにドロー

日付は早い順に並べる。店舗IDも数字の早い順から並べる。商品IDはすべての店舗が同じ商品数を売っているわけではないことを考慮して店舗IDごとにランダムで発生させた扱う商品数だけ1000個の商品群からランダムにとってくることにする。また、その扱う商品数と商品IDは年ごとに変化するものとする。

### 日付の作成

In [1]:
import datetime

In [2]:
start = datetime.datetime( 2013,1,1)
end = datetime.datetime(2016, 6,30)
td = end - start

In [3]:
days = [start + datetime.timedelta(days=i) for i in range(td.days+1)]

### 店舗IDと商品ID

In [4]:
import random
import numpy as np
from itertools import chain
from itertools import compress
num_shop = 100
num_prod = 1000
years = [2013, 2014, 2015, 2016]

In [5]:
prod_variation = np.random.randint(0, num_prod, (num_shop, len(years)))

In [6]:
products = [sorted(random.sample(range(num_prod), j)) for i in prod_variation for j in i]

In [7]:
a = [[1,0,0,0], [0,1,0,0], [0,0,1,0],[0,0,0,1]]
b = {}
c = ["2013", "2014", "2015", "2016"]

for i in range(len(years)):
    # 商品ID
    d = [j == 1 for j in a[i]*num_shop]
    e = list(compress(products, d))
    f = list(chain.from_iterable(e))
    # 店舗ID
    g = [[k]*prod_variation[k, i] for k in range(num_shop)]
    h = list(chain.from_iterable(g))
    # 辞書で保存
    emp = np.empty((len(h), 2))
    emp[:,0] = h
    emp[:, 1] = f
    b[c[i]] = emp

### データの作成

In [55]:
n2013 = 0
n2014 = 0
n2015 = 0
n2016 = 0
for i in days:
    if i.year == 2013:
        n2013 += 1
    elif i.year == 2014:
        n2014 += 1
    elif i.year == 2015:
        n2015 += 1
    else:
        n2016 += 1

m2013 = b["2013"].shape[0]
m2014 = b["2014"].shape[0]
m2015 = b["2015"].shape[0]
m2016 = b["2016"].shape[0]
        
num_rows = n2013 * b["2013"].shape[0] + n2014 * b["2014"].shape[0] + n2015 * b["2015"].shape[0] + n2016 * b["2016"].shape[0]

In [41]:
day = []
month = []
year = []
for i in days:
    if i.year == 2013:
        day += [i.day]*(b["2013"].shape[0])
        month += [i.month]*(b["2013"].shape[0])
        year += [i.year]*(b["2013"].shape[0])
    elif i.year == 2014:
        day += [i.day]*(b["2014"].shape[0])
        month += [i.month]*(b["2014"].shape[0])
        year += [i.year]*(b["2014"].shape[0])
    elif i.year == 2015:
        day += [i.day]*(b["2015"].shape[0])
        month += [i.month]*(b["2015"].shape[0])
        year += [i.year]*(b["2015"].shape[0])
    elif i.year == 2016:
        day += [i.day]*(b["2016"].shape[0])
        month += [i.month]*(b["2016"].shape[0])
        year += [i.year]*(b["2016"].shape[0])

In [45]:
data = np.empty((num_rows, 7))
data[:, 0] = year
data[:, 1] = month
data[:, 2] = day

In [58]:
for i in range(n2013):
    data[i*m2013:(i+1)*m2013, 3:5] = b["2013"]
    
for i in range(n2014):
    data[i*m2014 + m2013:(i+1)*m2014 + m2013, 3:5] = b["2014"]

for i in range(n2015):
    data[i*m2015 + m2013 + m2014:(i+1)*m2015 + m2013 + m2014, 3:5] = b["2015"]
    
for i in range(n2016):
    data[i*m2016 + m2013 + m2014 + m2015:(i+1)*m2016 + m2013 + m2014 + m2015, 3:5] = b["2016"]

In [71]:
max_sales = 10000
sales = np.random.rand(num_rows) * max_sales

In [67]:
max_quant = 1000
quant = np.random.randint(max_quant, size = num_rows)

In [72]:
data[:, 5] = sales
data[:, 6] = quant

### CSVに出力

In [73]:
import pandas as pd
df = pd.DataFrame(data, columns = ["year", "month", "day", "shop", "product", "sales", "quantity"])

In [75]:
df.head()

,year,month,day,shop,product,sales,quantity
0,2013.0,1.0,1.0,0.0,1.0,6979.236798,22.0
1,2013.0,1.0,1.0,0.0,2.0,1301.173915,940.0
2,2013.0,1.0,1.0,0.0,3.0,5165.175784,325.0
3,2013.0,1.0,1.0,0.0,4.0,5712.620137,327.0
4,2013.0,1.0,1.0,0.0,7.0,2987.072767,392.0


In [76]:
df.to_csv("test_data.csv")

csvのサイズは3.9GB

## 課題2 : 集計

### 月別、店舗別、総売上

In [91]:
df_grouped = df.groupby([df["year"], df["month"], df["shop"]])["sales"].sum()

In [97]:
df_grouped

year    month  shop
2013.0  1.0    0.0     4.261199e+07
               1.0     1.358965e+08
               2.0     5.159386e+07
               3.0     7.613543e+07
               4.0     8.446950e+07
               5.0     8.683432e+07
               6.0     3.332490e+07
               7.0     8.695514e+07
               8.0     1.540383e+07
               9.0     1.250186e+08
               10.0    5.215555e+07
               11.0    7.805937e+07
               12.0    1.213240e+08
               13.0    3.782665e+07
               14.0    5.709550e+07
               15.0    8.353277e+07
               16.0    7.671999e+07
               17.0    8.515109e+07
               18.0    8.065448e+07
               19.0    2.148450e+07
               20.0    1.150400e+08
               21.0    5.236692e+07
               22.0    2.653103e+07
               23.0    8.141133e+07
               24.0    4.956086e+07
               25.0    3.781399e+07
               26.0    5.819882e+07
        

### 月別、商品別、総売上

In [81]:
df_grouped2 = df.groupby([df["year"], df["month"], df["product"]])["sales"].sum()

In [96]:
df_grouped2

year    month  product
2013.0  1.0    0.0        7.347680e+06
               1.0        7.237888e+06
               2.0        7.690499e+06
               3.0        8.181586e+06
               4.0        7.886605e+06
               5.0        7.051877e+06
               6.0        7.718823e+06
               7.0        7.461314e+06
               8.0        6.477325e+06
               9.0        7.203023e+06
               10.0       7.437763e+06
               11.0       8.674822e+06
               12.0       7.142902e+06
               13.0       7.968757e+06
               14.0       7.056432e+06
               15.0       7.364865e+06
               16.0       8.104180e+06
               17.0       8.096537e+06
               18.0       7.418729e+06
               19.0       6.756734e+06
               20.0       7.559442e+06
               21.0       6.917765e+06
               22.0       8.598803e+06
               23.0       7.565216e+06
               24.0       7.299466e+06
  

### 日別、商品別、平均価格

In [87]:
df_grouped3 = df.groupby([df["year"], df["month"], df["day"], df["product"]])["sales", "quantity"].sum()

In [95]:
df_grouped3.assign(average = df_grouped3["sales"]/df_grouped3["quantity"])

sales    quantity    average
year   month day  product                                     
2013.0 1.0   1.0  0.0      2.581086e+05     24034.0  10.739313
                  1.0      2.524242e+05     25152.0  10.035949
                  2.0      2.595003e+05     26644.0   9.739539
                  3.0      2.231518e+05     20109.0  11.097108
                  4.0      2.711302e+05     23783.0  11.400167
                  5.0      2.120340e+05     19654.0  10.788338
                  6.0      2.664247e+05     28524.0   9.340369
                  7.0      2.472771e+05     24018.0  10.295492
                  8.0      2.131331e+05     26934.0   7.913163
                  9.0      1.647702e+05     19964.0   8.253366
                  10.0     2.422853e+05     22723.0  10.662557
                  11.0     2.469393e+05     26325.0   9.380410
                  12.0     2.129874e+05     19043.0  11.184549
                  13.0     2.412458e+05     21611.0  11.163101
                  14.0     2.225897e+05     25612.0   8.690838
                  15.0     3.180839e+05     29199.0  10.893657
                  16.0     2.481722e+05     28779.0   8.623378
                  17.0     1.864147e+05     19497.0   9.561199
                  18.0     2.387171e+05     23414.0  10.195487
                  19.0     2.161934e+05     24982.0   8.653967
                  20.0     2.467871e+05     22088.0  11.172905
                  21.0     2.196018e+05     25939.0   8.466087
                  22.0     2.318081e+05     24824.0   9.338063
                  23.0     1.810711e+05     21964.0   8.243993
                  24.0     2.469616e+05     27418.0   9.007280
                  25.0     2.349179e+05     23385.0  10.045664
                  26.0     2.494662e+05     24468.0  10.195611
                  27.0     2.825930e+05     26129.0  10.815303
                  28.0     2.421014e+05     21432.0  11.296257
                  29.0     2.491128e+05     20586.0  12.101078
...                                 ...         ...        ...
2016.0 6.0   1.0  0.0      2.540861e+08  25414304.0   9.997760
             2.0  0.0      2.553615e+08  25385342.0  10.059407
             3.0  0.0      2.535893e+08  25404886.0   9.981911
             4.0  0.0      2.536081e+08  25386033.0   9.990063
             5.0  0.0      2.541218e+08  25408605.0  10.001407
             6.0  0.0      2.540618e+08  25383065.0  10.009107
             7.0  0.0      2.541401e+08  25362960.0  10.020128
             8.0  0.0      2.530960e+08  25575722.0   9.895946
             9.0  0.0      2.543951e+08  25332141.0  10.042383
             10.0 0.0      2.555209e+08  25247012.0  10.120836
             11.0 0.0      2.548114e+08  25286369.0  10.077028
             12.0 0.0      2.537689e+08  25401872.0   9.990167
             13.0 0.0      2.545906e+08  25367885.0  10.035942
             14.0 0.0      2.550094e+08  25403632.0  10.038303
             15.0 0.0      2.531644e+08  25398352.0   9.967750
             16.0 0.0      2.539720e+08  25286366.0  10.043833
             17.0 0.0      2.541337e+08  25278462.0  10.053369
             18.0 0.0      2.545341e+08  25361999.0  10.036043
             19.0 0.0      2.536136e+08  25252742.0  10.043013
             20.0 0.0      2.541629e+08  25267808.0  10.058762
             21.0 0.0      2.552794e+08  25400664.0  10.050108
             22.0 0.0      2.541291e+08  25451033.0   9.985020
             23.0 0.0      2.543287e+08  25348834.0  10.033152
             24.0 0.0      2.534058e+08  25465120.0   9.951093
             25.0 0.0      2.530042e+08  25439570.0   9.945301
             26.0 0.0      2.540801e+08  25302382.0  10.041748
             27.0 0.0      2.539180e+08  25356953.0  10.013741
             28.0 0.0      2.533472e+08  25460510.0   9.950592
             29.0 0.0      2.537158e+08  25467974.0   9.962150
             30.0 0.0      2.534879e+08  25415074.0   9.973921

[373904 rows x 3 columns]